In [ ]:
aws s3 cp s3://ghgc-data-store-develop/experiment.geojson experiment.geojson

In [47]:
import geopandas as gpd
import pandas as pd
gdf_line = gpd.read_file("al022024_best_track/AL022024_lin.shp")
gdf_pt = gpd.read_file("al022024_best_track/AL022024_pts.shp")
gdf_wind  = gpd.read_file("al022024_best_track/AL022024_windswath.shp")

In [48]:
gdf_wind.columns, gdf_line.columns, gdf_pt.columns

(Index(['RADII', 'STORMID', 'BASIN', 'STORMNUM', 'ADVNUM', 'STARTDTG', 'ENDDTG',
        'geometry'],
       dtype='object'),
 Index(['STORMNUM', 'STORMTYPE', 'SS', 'geometry'], dtype='object'),
 Index(['STORMNAME', 'DTG', 'YEAR', 'MONTH', 'DAY', 'HHMM', 'MSLP', 'BASIN',
        'STORMNUM', 'STORMTYPE', 'INTENSITY', 'SS', 'LAT', 'LON', 'geometry'],
       dtype='object'))

In [49]:
def assign_category(row):
    if row['SS'] == 1.0 and row['STORMTYPE'] == 'HU':
        return 'C1'
    elif row['SS'] == 2.0 and row['STORMTYPE'] == 'HU':
        return 'C2'
    elif row['SS'] == 3.0 and row['STORMTYPE'] == 'HU':
        return 'C3'
    elif row['SS'] == 4.0 and row['STORMTYPE'] == 'HU':
        return 'C4'
    elif row['SS'] == 5.0 and row['STORMTYPE'] == 'HU':
        return 'C5'
    else:
        return row['STORMTYPE']




In [50]:
# Put datetime
gdf_pt['datetime'] = pd.to_datetime(
    gdf_pt[['YEAR', 'MONTH', 'DAY', 'HHMM']]
    .astype(int)
    .astype(str)
    .apply(lambda x: f"{x.YEAR}-{x.MONTH.zfill(2)}-{x.DAY.zfill(2)} {x.HHMM.zfill(4)}", axis=1), 
    format='%Y-%m-%d %H%M'
)
gdf_pt['datetime'] = gdf_pt['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
gdf_pt['SS'] = gdf_pt.apply(lambda x: 0.5 if x['STORMTYPE'] == 'TS' else x['SS'], axis=1)
gdf_pt['CATEGORY'] =  gdf_pt['STORMTYPE']
gdf_pt['CATEGORY'] =  gdf_pt.apply(assign_category, axis=1)

gdf_wind['datetime'] = pd.to_datetime(gdf_wind['STARTDTG'], format='%Y%m%d%H')
gdf_wind['datetime'] = gdf_wind['datetime'].dt.strftime('%Y-%m-%dT%H:%M:%SZ')
gdf_wind['STORMNUM'] = 2.0

gdf_line['SS'] = gdf_line.apply(lambda x: 0.5 if x['STORMTYPE'] == 'TS' else x['SS'], axis=1)

In [52]:
gdf_pt[["datetime", "STORMNAME", "SS", "STORMTYPE", "MSLP", "INTENSITY","CATEGORY" ,"geometry"]].to_file("path_point_cyclone_beryl.geojson",  driver="GeoJSON")

In [23]:
gdf_wind[["RADII", "geometry"]].to_file("wind_polygon_cyclone_beryl.geojson",  driver="GeoJSON")

In [24]:
gdf_line[['STORMTYPE', 'SS', 'geometry']].to_file("path_line_cyclone_beryl.geojson",  driver="GeoJSON")

In [45]:
gdf_pt[['SS','STORMTYPE','category']]

,SS,STORMTYPE,category
0,0.0,TD,TD
1,0.0,TD,TD
2,0.5,TS,TS
3,0.5,TS,TS
4,0.5,TS,TS
5,0.5,TS,TS
6,1.0,HU,C1
7,2.0,HU,C2
8,3.0,HU,C3
9,4.0,HU,C4


In [53]:
gdf_pt.shape

(54, 17)

In [13]:
import requests
import time

def measure_avg_response_time(url, num_tests=100):
    total_time = 0
    success_count = 0
    
    for _ in range(num_tests):
        start_time = time.time()
        try:
            response = requests.get(url, timeout=10)  # Set timeout to avoid hanging requests
            end_time = time.time()
            total_time += (end_time - start_time)
            success_count += 1
        except requests.exceptions.RequestException as e:
            print(f"Error with {url}: {e}")

    avg_time = total_time / success_count if success_count else float('inf')
    return avg_time, success_count

# api_1 = "https://pl6gha7jt0.execute-api.us-west-2.amazonaws.com/collections/public.noaa_glm_flask_surface_abp_brazil_co2_event"
# api_2 = "https://pl6gha7jt0.execute-api.us-west-2.amazonaws.com/collections/public.n_noaa_glm_flask_surface_abp_brazil_co2_event"
api_1 = "https://dev.ghg.center/api/features/collections/public.noaa_glm_flask_surface_abp_brazil_co2_event"
api_2 = "https://dev.ghg.center/api/features/collections/public.n_noaa_glm_flask_surface_abp_brazil_co2_event"

avg_time_1, success_1 = measure_avg_response_time(api_1)
avg_time_2, success_2 = measure_avg_response_time(api_2)

print(f"API 1 Average Response Time: {avg_time_1:.4f} seconds | Successful Requests: {success_1}/20")
print(f"API 2 Average Response Time: {avg_time_2:.4f} seconds | Successful Requests: {success_2}/20")

if avg_time_1 < avg_time_2:
    print("API 1 is faster on average.")
elif avg_time_1 > avg_time_2:
    print("API 2 is faster on average.")
else:
    print("Both APIs have the same average response time.")


API 1 Average Response Time: 0.0558 seconds | Successful Requests: 100/20
API 2 Average Response Time: 0.0611 seconds | Successful Requests: 100/20
API 1 is faster on average.


In [ ]:
API 1 Average Response Time: 0.1709 seconds | Successful Requests: 10/20
API 2 Average Response Time: 0.1734 seconds | Successful Requests: 10/20
API 1 is faster on average.

In [4]:
list1 = [1, 2, 3]
list2 = [4, 5, 6]
list1.append(list2)
print(list1) 

[1, 2, 3, [4, 5, 6]]
